In [1]:
# Load package
suppressPackageStartupMessages({
  library(splatter)
  library(scater)
  library(Matrix)
})

# Create mock data
set.seed(1)

In [10]:
# Set simulation settings
# Create params object
params <- newSplatParams()
nGroups <- 5
nBatches <- 2
batchCells <- 2000
group.prob <- rep(1/5, nGroups)
de.prob <- 0.01
# Set params values
params <- setParams(params, 
                    update = list(
           nGenes = nGenes, 
                        batchCells = rep(batchCells, nBatches),
                        batch.facLoc = 2,
                        batch.facScale = 0.5**2,
                        group.prob = group.prob,
                        de.prob = de.prob,
                        de.downProb = 0,
                        de.facLoc = 2,
                        de.facScale = 0.5**2
                    )
                   )

# sim <- splatSimulate(params, method='groups')
# https://github.com/Oshlack/splatter/issues/57

In [3]:
count_simulator <- function(
    nGenes,
    batchCells,
    group.prob,
    de.prob,
    de.facLoc
    ){
    
    params <- newSplatParams()
    params <- setParams(
                    params, 
                    update = list(
                        nGenes = nGenes, 
                        batchCells = batchCells,
                        batch.facLoc = 2,
                        batch.facScale = 0.5**2,
                        group.prob = group.prob,
                        de.prob = de.prob,
                        de.downProb = 0,
                        de.facLoc = de.facLoc,
                        de.facScale = 0.5**2
                    )
                )
    
    print(params)
    
    sim <- splatSimulate(params, method='groups', verbose=FALSE)
    counts(sim) <- as(counts(sim), "sparseMatrix") # matrix to sparse
    sim
}

# Set simulation parameters   
de.facLocs <- c(1.2, 1.5, 2, 2.5, 3, 4)
nGroups <- 5
nBatches <- 2
batchCells <- 2000
nGenes <- 15000

# Run simulation
for (de.facLoc in de.facLocs){
    # Create simulated count
    sce <- count_simulator(
        nGenes = nGenes,
        batchCells = rep(batchCells, nBatches),
        group.prob = rep(1/5, nGroups),
        de.prob = 3000/nGenes/nGroups,
        de.facLoc = de.facLoc
        )
    
    # is the gene DEG?
    colDEG <- paste0('DEFacGroup', 1:nGroups)
    colBatch <- paste0()
    rowData(sce)$isDEG <- as.logical(rowSums(as.matrix(rowData(sce)[,colDEG]) >= de.facLoc))
    
    # Save simulatoin results
    filenamesuffix <- paste(nGroups, nBatches, batchCells, de.facLoc, sep='_')
    writeMM(counts(sce), paste0('simdata/', filenamesuffix, '.mtx'))
    write.csv(colData(sce), paste0('simdata/meta_', filenamesuffix, '.csv'))
    write.csv(rowData(sce), paste0('simdata/gene_', filenamesuffix, '.csv'))
    
}

A Params object of class SplatParams 
Parameters can be (estimable) or [not estimable], 'Default' or  'NOT DEFAULT' 
Secondary parameters are usually set during simulation

Global: 
(GENES)  (CELLS)   [SEED] 
  15000     4000   452737 

29 additional parameters 

Batches: 
    [BATCHES]  [BATCH CELLS]     [LOCATION]        [SCALE]       [Remove] 
            2     2000, 2000              2           0.25          FALSE 

Mean: 
 (Rate)  (Shape) 
    0.3      0.6 

Library size: 
(Location)     (Scale)      (Norm) 
        11         0.2       FALSE 

Exprs outliers: 
(Probability)     (Location)        (Scale) 
         0.05              4            0.5 

Groups: 
              [GROUPS]           [GROUP PROBS] 
                     5  0.2, 0.2, 0.2, 0.2,... 

Diff expr: 
[PROBABILITY]    [DOWN PROB]     [LOCATION]        [SCALE] 
         0.04              0            1.2           0.25 

BCV: 
(Common Disp)          (DoF) 
          0.1             60 

Dropout: 
    [Type]  (Midpoi